In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import defaultdict


class QTrader(object):
    def __init__(self, sym_data, eta=1, transaction_cost=0.005, position=10):
        self.sym_data = sym_data
        self.stock_data = sym_data#pd.merge(pd.read_csv('./tbill.csv', index_col='Date'), pd.read_csv('./^GSPC.csv', index_col='Date'), right_index=True, left_index=True).sort_index()

        # These are price returns and the weekly returns for TBills (3 months)
        self.returns = pd.DataFrame({
                                        'stocks': self.stock_data['Close'].rolling(window=2, center=False).apply(lambda x: x[1] / x[0] - 1),
                                        'tbills': (self.stock_data['Close'].std() / 100 + 1) ** (1/52) - 1,
                                    }, index=self.stock_data.index)

        self.returns['risk_adjusted'] = self.returns.stocks - self.returns.tbills
        self.returns['risk_adjusted_moving'] = self.returns.risk_adjusted.rolling(window=12).apply(lambda x: x.mean())
        self.returns['risk_adjusted_stdev'] = self.returns.risk_adjusted.rolling(window=12).apply(lambda x: x.std())
        self.returns['risk_adjusted_high'] = self.returns.risk_adjusted_moving + 1.5 * self.returns.risk_adjusted_stdev
        self.returns['risk_adjusted_low'] = self.returns.risk_adjusted_moving - 1.5 * self.returns.risk_adjusted_stdev
        self.returns['state'] = (self.returns.risk_adjusted > self.returns.risk_adjusted_high).astype('int') - \
                                (self.returns.risk_adjusted < self.returns.risk_adjusted_high).astype('int') # pd.qcut(self.returns.sharpe_moving, 10, labels=range(10))
    def graph_portfolios(self):

        midpoint = int(len(self.returns.index) / 2)
        training_indexes = self.returns.index[:midpoint] 
        testing_indexes = self.returns.index[midpoint:]

        portfolios = pd.DataFrame({
            'buy_and_hold': self.buy_and_hold(testing_indexes),
            'buy_tbills': self.buy_tbills(testing_indexes),
            'random': self.random(testing_indexes),
            'qtrader': self.q_holdings(training_indexes, testing_indexes)
            }, index=testing_indexes)

        portfolio_values = pd.DataFrame({
                'buy_and_hold': self.evaluate(portfolios.buy_and_hold),
                'buy_tbills': self.evaluate(portfolios.buy_tbills),
                'random': self.evaluate(portfolios.random),
                'qtrader': self.evaluate(portfolios.qtrader)
            }, index=testing_indexes)
        
        portfolio_values.plot()

        plt.annotate("Buy and hold sharpe: {}\n QTrader sharpe: {}".format(self.sharpe(portfolios.buy_and_hold), self.sharpe(portfolios.qtrader)), xy=(0.25, 0.95), xycoords='axes fraction')
        
        plt.show()
        
    def buy_and_hold(self, dates):
        return pd.Series(1, index=dates)

    def buy_tbills(self, dates):
        return pd.Series(0, index=dates)

    # This will go long or short or do nothing randomly
    def random(self, dates):
        return pd.Series(np.random.randint(-1, 2, size=len(dates)), index=dates)

    def calc_returns(self, holdings):
        return pd.Series(self.returns.tbills + holdings * self.returns.risk_adjusted, index=holdings.index)

    def evaluate(self, holdings):
        return (self.calc_returns(holdings) + 1).cumprod()

    def state(self, first_moment, second_moment):
        return first_moment + second_moment * 10

    def q_holdings(self, training_indexes, testing_indexes):
        factors = pd.DataFrame({'action': 0, 'reward': 0, 'state': 0}, index=training_indexes)

        # Initialize Q matrix
        q = {0: {1:0, 0:0, -1:0}}
        
        # For Dyna-Q
        T = np.zeros((3, 3, 3)) + 0.00001
        R = np.zeros((3,3))

        # Episodes
        for i in range(100):
            last_row, last_date = None, None

            for date, row in factors.iterrows():
                return_data = self.returns.loc[date]
                if return_data.state not in q:
                    q[return_data.state] = {1: 0, 0:0, -1:0}

                if last_row is None or np.isnan(return_data.state):
                    state = 0
                    reward = 0
                    action = 0
                else:
                    state = int(return_data.state)
                    if random.random() > 0.001:
                        action = max(q[state], key=q[state].get)
                    else:
                        action = random.randint(-1,1)

                    reward = last_row.action * (return_data.stocks - return_data.tbills)

                    alpha = 1
                    discount = 0.9

                    factors.loc[date, 'reward'] = reward
                    factors.loc[date, 'action'] = action
                    factors.loc[date, 'state'] = return_data.state

                    update = alpha * (factors.loc[date, 'reward'] + discount * max(q[row.state].values()) - q[state][action])
                    if not np.isnan(update):
                        q[state][action] += update

                    # Dyna
                    action_idx = int(last_row.action+1)
                    state_idx = int(last_row.state+1)
                    new_state_idx = int(state+1)

                    T[state_idx][action_idx][new_state_idx] += 1
                    R[state_idx][action_idx] = (1 - alpha) * R[state_idx][action_idx] + alpha * reward

                last_date, last_row = date, factors.loc[date]

            for j in range(100):
                state_idx = random.randint(0,2)
                action_idx = random.randint(0,2)
                new_state = np.random.choice([-1, 0, 1], 1, p=T[state_idx][action_idx]/T[state_idx][action_idx].sum())[0]
                r = R[state_idx][action_idx]
                q[state][action] += alpha * (r + discount * max(q[new_state].values()) - q[state][action])

            sharpe = self.sharpe(factors.action)

            if sharpe > 0.20:
                break
            print("For episode {} we get an internal sharpe ratio of {}".format(i, self.sharpe(factors.action)))

        testing = pd.DataFrame({'action': 0, 'state': 0}, index=testing_indexes)
        testing['state'] = self.returns.loc[testing_indexes, 'state']
        testing['action'] = testing['state'].apply(lambda state: max(q[state], key=q[state].get))

        print(self.sharpe(testing.action))

        return testing.action

    def discretize(self, number, steps):
        return 0

    def sharpe(self, holdings):
        returns = holdings * (self.returns.stocks - self.returns.tbills)

        return np.nanmean(returns) / np.nanstd(returns)

    


In [3]:

import datetime
from time import sleep
from binance.client import Client
import pandas as pd
import datetime
import time
import numpy as np
import os
import sys
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from statsmodels.tsa.stattools import coint

#from crontab import CronTab
from safe_schedule import SafeScheduler
#add your api keys here
api_key = 'XuozVlFg2WYVemwmHMqCK9j5jSYIV1z3MoDAXY76X2JiCbbgcw7dGxvE0eGVsqhH'
secret_key = 'WyWBCdczLCFvV8fKy16R5dOwhhjK31T7ZrW0cplIzjrvmlSPh5E1KEeAEh5c9I6s'

#Open location of recorded buy/sell walls
bw_location = '/home/me/Documents/BWA/data/watcher/buywalls/'
sw_location = '/home/me/Documents/BWA/data/watcher/sellwalls/'
save_port = '/githome/me/Documents/BWA/data/bw_portfolios'

#Open binance client
client = Client(api_key, secret_key)

#get current time and format to compare with data
current_milli_time = lambda: int(round(time.time() * 1000))


def get_data(sym,prev_k,k,candle_k='5m'):
    """
    Function downloads hist data from binance API with given timestamps
    """
    #previous kline interval
    prev_k = str(prev_k)
    #current kline interval
    k = str(k)
    #download historical data
    data = client.get_historical_klines(sym,interval= candle_k,start_str =int(prev_k),end_str = int(k))
    data = pd.DataFrame(data,columns=['Open time','Open','High','Low','Close','Volume','Close time','Quote volume','Num trades', 'btc buys', 'coin buys', 'ignore'])
    
    # convert from millesecond time to datetime for potential resampling and readability
    data['Open time'] = data['Open time']/1000
    data['Open time'] = data['Open time'].astype('datetime64[s]')
    data = data.set_index('Open time')
    data = data[['Open','High','Low','Close','Close time','Volume']].astype(float)
    return data

def normalize(data,column="Close",deci = False):
    """
    Function for simple time series normalization 
    """
    if(deci==True):
        norm_close =  data[column].divide(data[column].iloc[0])-1
    else:
        norm_close =  data[column].divide(data[column].iloc[0])
        
    return norm_close


def to_dt(v,is_milli=True):
    """
    Function to convert timestamp to datetime object
    has parameter for millisecond uts timestamp
    """
    if(is_milli):
        v = datetime.datetime.fromtimestamp(int(v)/1000)
    else:
        v = datetime.datetime.fromtimestamp(int(v))
    return v

def get_freq_sig(loc):
    """
    Function loads all signal csv files and concatenates them into a signle dataframe
    """
    # cd cmd
    os.chdir(bw_location)
    #sorting by file creation (could use normal sort as filenames are timestamps)
    files = filter(os.path.isfile,os.listdir(bw_location))
    files =  [int(f) for f in files]
    files = list(files)
    #Create frequency signal dataframe
    fs_df = pd.DataFrame(files)
    #sort again
    fs_df = fs_df[0].sort_values()
    #starttime,endtime
    st,end = to_dt(fs_df.iloc[0]),to_dt(fs_df.iloc[-1])
    bw_freq = []
    
    f_len = int(len(os.listdir(loc)))
    for c,i in enumerate(os.listdir(loc)):
        try:
            curr = pd.read_csv(loc+i)
        except Exception as e:
            print(i,e)
 
        curr.columns = ['Coin','Close','Profit','Loss','Date']  
        bw_freq.append(curr)
        if((c/f_len)%10==0):
            pct = str(int((c/f_len)*100))
            print("{}% of files loaded".format(pct))
    bw_freq = pd.concat(bw_freq)
    bw_freq = bw_freq[['Date','Coin','Close']]
    bw_freq.columns = ['Date','Coin','Close']
    bw_freq = bw_freq.sort_values(['Date'],ascending=False)
    #print(all_w.head(), len(all_w))
    #bw_freq['all_freq'] = bw_freq.groupby('Coin')['Coin'].transform('count')
    #bw_freq = bw_freq.sort_values(['Date'],ascending=False)    
    print(st,end)
    print(to_dt(now), now)
    print(fs_df.head())
    return bw_freq

def clean_fs(fs):
    """
    Function to clean frequency signal dataframe
    """
    fs =fs.drop_duplicates()
    fs = fs.sort_values(['Date'],axis=0)
    fs['Date_m'] = fs['Date']
    fs['Date'] = fs['Date']/1000
    fs['Date'] = fs['Date'].astype('datetime64[s]')

    rolling_f = {}
    f_col = []
    """
    for i in fs.iterrows():
        coin = i[1][1]
        if(coin in rolling_f.keys()):
            rolling_f[coin] += 1
            f_col.append(rolling_f[coin])
        else:
            rolling_f[coin] = 1
            f_col.append(rolling_f[coin])
    print(len(fs),len(f_col))
    fs['rolling_freq'] = f_col
    """    
    return fs
def make_all_coins(cgroups):
    all_coins = pd.DataFrame()
    for sym in coins:
        coin = cgroups.get_group(sym)

        #the secret sauce, extracting value from our data
        #for this example im using the time difference between the signal event occurence to determine enter/exits
        t_dif = []
        for i in range(0,len(coin)-1):
            t1 = coin['Date_m'].iloc[i]
            t2 = coin['Date_m'].iloc[i+1]
            td = (t2-t1)
            #if(td) signal logic
            t_dif.append(td)
        t_dif.append(0)
        #print(t_dif==coin['t_diff'])
        coin = coin.assign(t_dif = t_dif)
        coin.t_dif = coin.t_dif.shift(1)

        coin = coin.fillna(method='bfill')
        coin.t_dif = coin.t_dif.pct_change()
        coin = coin[['Close','t_dif','Date_m']].copy()
        coin.Close = normalize(coin)
        coin.fillna(method='bfill')

        #make copy w/ no index for resample and hist data sync
        #create daily signals from intraday signal dataset
        dc = coin.reset_index().copy()
        dc.index = dc.Date
        #Daily resample
        daily_coin = dc.resample('D').sum()

        #daily resampled date index
        dates = list(daily_coin.index)
        print(sym)
        print(dates[0].strftime('%Y-%m-%d'),"   |   ",dates[-1].strftime('%Y-%m-%d'))
        prev_k,k = int(dates[0].timestamp()*1000), int(dates[-1].timestamp()*1000)

        #Download historical data
        c_hist = get_data(sym,prev_k,k,candle_k='1d')
        c_hist['Entry'] = to_dt(coin.Date_m.iloc[-1])
        #extract signal for resampled signal dataframe and create signal column (-1,1) ...maybe (-1,0,1)
        c_hist['Close_diff'] = c_hist['Close'].diff().apply(np.sign)

        # log of 5 day moving average of volume

        c_hist['5dlv'] = c_hist.Volume.rolling(5).mean().apply(np.log) 

        # daily volume vs. 200 day moving average

        c_hist['v_3v'] = c_hist.Volume/ c_hist.Volume.rolling(3).mean()-1
        c_hist['vpct'] =  c_hist.Volume.pct_change()
        c_hist['3vpct'] = c_hist['vpct'].rolling(3).sum()


        c_hist['bw_sig'] = daily_coin.t_dif
        c_hist['sig3'] = c_hist["bw_sig"].apply(np.sign)
        c_hist['sig3'] = c_hist['sig3'].replace(0,-1)
        c_hist = c_hist.fillna(0) 
        c_hist["daily_returns"] = (c_hist['Close']/c_hist['Open'])-1
        """
        #Visualize trading range and signal
        c_hist2 = c_hist[["Close","sig3"]].copy()
        c_hist2.Close = normalize(c_hist2,deci=True)
        c_hist2["High"] = normalize(c_hist,column="High",deci=True)
        c_hist2["Low"] = normalize(c_hist,column="Low",deci=True)
        c_hist2.plot(figsize=(8,5))
        #plt.savefig("/home/me/Pictures/{}range_signal.png".formate(sym))

        c_hist[['sig3','Close_diff']].plot(grid=True,figsize=(8,5))
        """


        c_hist['Market Returns'] = np.log(c_hist['Close'] / c_hist['Close'].shift(1))
        c_hist['Strategy Returns'] = c_hist['Market Returns'] * c_hist['sig3'].shift(1)
        c_hist['Coin'] = sym
        c_hist = c_hist.reset_index()
        c_hist = c_hist.set_index(['Open time','Coin'])

        all_coins = pd.concat([all_coins,c_hist])

    #dts is dates for resampled df
    dts = all_coins.index.get_level_values(0)

    fol = dts[0].strftime('%Y-%m-%d')+"_"+dts[-1].strftime('%Y-%m-%d')
    print(fol)
    #save all_coin to csv
    fname = '/home/me/Documents/BWA/data/'
    if os.path.exists(fname) == False:
        os.mkdir(fname)
        all_coins.to_csv(fname+"all_coin.csv")
    return all_coins

def unix_time_millis(dt):
    """
    Function to convert unix time to millesecond
    """
    epoch = datetime.datetime.utcfromtimestamp(0)
    return (dt - epoch).total_seconds() * 1000.0



def interval(intv,st,end):
    """
    Function to resample frequency dataframe to specified frequency
    
    """
    intv_list = []
    out = pd.DataFrame()
    prev = st
    while(prev<end):
        curr = prev+int(86400000*intv)
        n = to_dt(curr).strftime('%Y-%m-%d')
        p = to_dt(prev).strftime('%Y-%m-%d')
        
        print(p,"  |  ",n)
        
        prev = curr
        p = datetime.datetime.strptime(p, '%Y-%m-%d')
        n = datetime.datetime.strptime(n, '%Y-%m-%d')
        intv_list.append(p)
        data = fs_c.loc[p:n]
        #print(data.sort_values('rolling_freq',ascending=False))
        out = pd.concat([out,data],axis=0)
        
    return out.sort_index(),intv_list
    #files.sort(key=lambda x: os.path.getmtime(x))


In [5]:
#get freq. signal df
freq_loc = '/home/me/Documents/BWA/data/freq_concat.csv'
fs = pd.read_csv(freq_loc)
#clean freq. signal df
fs_c = clean_fs(fs)
#multilevel indexing for interval tracking
fs_c = fs_c.set_index(['Date','Coin'])
#print(fs_c.head(3))
#print(fs_c.tail(3))
#Create start and end for interval dataframe 
st,end = fs_c.index[0][0],fs_c.index[-1][0]
st,end = unix_time_millis(st),unix_time_millis(end)

#30d interval for portfolio generation
intv = 30
#make multiindex frequency dataframe 
#works on fs_c
freq_df,intv_list = interval(intv,st,end)
#group by Coin
cgroups = freq_df.groupby('Coin')
#make list of all syms/dates in dataframe.... the index
coins = list(freq_df.index.get_level_values('Coin').drop_duplicates())
dates = list(freq_df.index.get_level_values('Date').drop_duplicates())


#test print of grouped dataframe
coin = cgroups.get_group(coins[0])
fname = '/home/me/Documents/BWA/data/'
update = input("update all_coins?")
if update == 'y':
    all_coins = make_all_coins(cgroups)
else:
    all_coins = pd.read_csv(fname+'all_coins.csv')

2018-08-24   |   2018-09-23
2018-09-23   |   2018-10-23
2018-10-23   |   2018-11-21
2018-11-21   |   2018-12-21
2018-12-21   |   2019-01-20
2019-01-20   |   2019-02-19
update all_coins?y
DASHBTC
2018-08-24    |    2019-01-20
ENGBTC
2018-08-24    |    2019-01-21
THETABTC
2018-08-24    |    2019-01-21
APPCBTC
2018-08-24    |    2019-01-21
GVTBTC
2018-08-24    |    2019-01-20
OAXBTC
2018-08-24    |    2019-01-21
AIONBTC
2018-08-24    |    2019-01-07
HCBTC
2018-08-24    |    2019-01-21
WINGSBTC
2018-08-24    |    2019-01-21
POLYBTC
2018-08-24    |    2019-01-21
ARDRBTC
2018-08-24    |    2019-01-19
ZENBTC
2018-08-24    |    2019-01-21
CMTBTC
2018-08-24    |    2019-01-21
IOTABTC
2018-08-24    |    2019-01-21
NEBLBTC
2018-08-24    |    2019-01-21
ETHBTC
2018-08-24    |    2019-01-20
ICXBTC
2018-08-24    |    2019-01-20
LOOMBTC
2018-08-24    |    2019-01-21
NANOBTC
2018-08-24    |    2019-01-21
REPBTC
2018-08-24    |    2019-01-21
MODBTC
2018-08-24    |    2019-01-21
LSKBTC
2018-08-24    |  

## Daily resampling for signal/event generation

### creates a dataframe for all coins w/ signals for trading/backtesting
Makes a single csv w/ all coins and multilevel indexing by portfolio time
https://alphascientist.com/feature_engineering.html

c_hist is feature df

# Generate portfolios

In [7]:
last_n_hist = pd.DataFrame()
portfolio = pd.DataFrame()
dates = all_coins.index.get_level_values('Open time').drop_duplicates()
coins = all_coins.index.get_level_values('Coin')
last_dates = dates[-5:]
for day in dates[-5:]:
    curr = all_coins.loc[day].sort_values(['bw_sig','vpct'],ascending=[False,False]).iloc[:10].copy()
    curr = curr[['Open','Close','bw_sig','sig3','Entry']].copy()
    curr['Date'] = day
    portfolio = pd.concat([portfolio,curr])
    

    #curr

    #print("\n",day,"\n",pt)
    #print(curr)
    
#all_coins.sort_values(["Open time","v_3v"]).iloc[-10:]
portfolio = portfolio.reset_index()
portfolio.set_index(['Date','Coin'])

Open     Close        bw_sig  sig3  \
Date       Coin                                               
2019-01-17 DOCKBTC   0.000003  0.000003  10352.470951   1.0   
           MTLBTC    0.000066  0.000065   1738.459728   1.0   
           GXSBTC    0.000146  0.000154   1700.082830   1.0   
           VIABTC    0.000083  0.000084   1506.548752   1.0   
           REQBTC    0.000006  0.000006   1118.935283   1.0   
           EOSBTC    0.000669  0.000685    518.807114   1.0   
           XRPBTC    0.000091  0.000090    517.718062   1.0   
           NXSBTC    0.000095  0.000096    375.685085   1.0   
           EDOBTC    0.000226  0.000228    293.986607   1.0   
           CVCBTC    0.000014  0.000015    293.710757   1.0   
2019-01-18 STEEMBTC  0.000094  0.000107   3023.244830   1.0   
           XLMBTC    0.000030  0.000029    704.976424   1.0   
           WINGSBTC  0.000021  0.000021    502.437339   1.0   
           WAVESBTC  0.000712  0.000703    499.127625   1.0   
           LOOMBTC   0.000013  0.000012    492.434622   1.0   
           TUSDBTC   0.000275  0.000277    436.429289   1.0   
           MODBTC    0.000085  0.000090    365.477338   1.0   
           QLCBTC    0.000007  0.000007    348.632179   1.0   
           SUBBTC    0.000013  0.000013    343.032808   1.0   
           ARKBTC    0.000111  0.000114    337.299359   1.0   
2019-01-19 POABTC    0.000008  0.000009   1394.264178   1.0   
           ENJBTC    0.000009  0.000009    828.164218   1.0   
           ELFBTC    0.000030  0.000031    692.083472   1.0   
           EOSBTC    0.000676  0.000667    566.214277   1.0   
           OSTBTC    0.000006  0.000006    564.035155   1.0   
           KMDBTC    0.000181  0.000188    524.501933   1.0   
           NXSBTC    0.000093  0.000098    511.823780   1.0   
           STORJBTC  0.000037  0.000041    489.942606   1.0   
           CMTBTC    0.000007  0.000008    363.540734   1.0   
           NEBLBTC   0.000316  0.000321    230.352789   1.0   
2019-01-20 ZILBTC    0.000006  0.000006   1796.781839   1.0   
           ENGBTC    0.000078  0.000078    784.482329   1.0   
           ETHBTC    0.033441  0.033214    642.478092   1.0   
           SYSBTC    0.000012  0.000012    451.861047   1.0   
           PPTBTC    0.000368  0.000362    448.405752   1.0   
           LSKBTC    0.000345  0.000343    248.787299   1.0   
           POWRBTC   0.000032  0.000032    241.102924   1.0   
           REPBTC    0.004681  0.005073    223.161616   1.0   
           NEBLBTC   0.000322  0.000313    216.409302   1.0   
           ENJBTC    0.000009  0.000009    214.268443   1.0   
2019-01-21 ASTBTC    0.000008  0.000008    471.230108   1.0   
           BCPTBTC   0.000009  0.000009    353.021443   1.0   
           IOTABTC   0.000083  0.000085    339.718227   1.0   
           QSPBTC    0.000005  0.000005    236.875428   1.0   
           BNTBTC    0.000154  0.000152    212.621153   1.0   
           PPTBTC    0.000363  0.000378    201.796643   1.0   
           ENGBTC    0.000078  0.000078    185.071636   1.0   
           KMDBTC    0.000182  0.000183    183.152433   1.0   
           CVCBTC    0.000014  0.000015    166.079402   1.0   
           MODBTC    0.000087  0.000090    162.336310   1.0   

                                      Entry  
Date       Coin                              
2019-01-17 DOCKBTC  2019-01-17 16:48:13.365  
           MTLBTC   2019-01-20 16:11:01.544  
           GXSBTC   2019-01-19 22:32:35.253  
           VIABTC   2019-01-20 03:55:36.968  
           REQBTC   2019-01-17 09:46:18.810  
           EOSBTC   2019-01-19 18:57:56.547  
           XRPBTC   2019-01-20 20:03:40.428  
           NXSBTC   2019-01-20 16:16:48.662  
           EDOBTC   2019-01-20 06:11:28.378  
           CVCBTC   2019-01-21 00:59:01.170  
2019-01-18 STEEMBTC 2019-01-18 11:08:14.080  
           XLMBTC   2019-01-18 16:21:13.865  
           WINGSBTC 2019-01-21 02:47:51.496  
           WAVESBTC 2019-01-21 09:42:08.306  
           L

In [8]:
today = portfolio.iloc[-10:].copy()
today.reset_index()
today = today.set_index("Coin")
today['Open'].astype(str)

Coin
ASTBTC       7.73e-06
BCPTBTC      9.25e-06
IOTABTC     8.345e-05
QSPBTC       4.68e-06
BNTBTC     0.00015396
PPTBTC      0.0003626
ENGBTC      7.834e-05
KMDBTC      0.0001824
CVCBTC      1.449e-05
MODBTC       8.71e-05
Name: Open, dtype: object

In [32]:

now = current_milli_time()

qt_rets = {}
today_port = pd.DataFrame()
for t in today.index:
    st,end = int(unix_time_millis(today.loc[t]['Entry'])),now
    t_hist = get_data(t,st,end,"5m")
    print('\n\n\n\n',t,today.loc[t]['Entry'],to_dt(now))
    print(t_hist.head())
    print("======================")
    qt = QTrader(t_hist)
    
    #print(qt.buy_and_hold(t_hist.index))
    qt_rets[t] = qt
    ast = qt_rets[t]
    #later replace with corr data
    midpoint = int(len(ast.returns.index) / 2)
    training_indexes = ast.returns.index[:midpoint] 
    testing_indexes = ast.returns.index[midpoint:]
    sym_trade = ast.q_holdings(training_indexes,testing_indexes)
    t_hist['Coin'] = t
    t_hist['qsig'] = sym_trade
    t_hist = t_hist.reset_index()
    t_hist = t_hist.set_index(['Open time','Coin'])
    today_port = pd.concat([today_port,t_hist])
    
fname = to_dt(now).strftime("%Y-%m-%d")+"_trades.csv"
today_port.to_csv("/home/me/Documents/BWA/data/trades/{}".format(fname))
    #qt.graph_portfolios()
    #ret = bollinger_strat(t_hist,250,1)
    #ret2 = mom_strat(t_hist.copy())
    #rets.append(ret)
    
    """ 
    #And iterate through them both, running the strategy function each time
    for window in windows:
        for std in stds:
            bollinger_strat(t_hist.copy(),window,std)
            print(window,std)
            plt.show()
    """
    #print(st,end)
    #returns[t] = ret
   





 ASTBTC 2019-01-21 07:40:16.855000 2019-01-23 00:48:27.967000
                         Open      High       Low     Close    Close time  \
Open time                                                                   
2019-01-21 07:45:00  0.000008  0.000008  0.000008  0.000008  1.548057e+12   
2019-01-21 07:50:00  0.000008  0.000008  0.000008  0.000008  1.548057e+12   
2019-01-21 07:55:00  0.000008  0.000008  0.000008  0.000008  1.548058e+12   
2019-01-21 08:00:00  0.000008  0.000008  0.000008  0.000008  1.548058e+12   
2019-01-21 08:05:00  0.000008  0.000009  0.000008  0.000009  1.548058e+12   

                       Volume  
Open time                      
2019-01-21 07:45:00  248210.0  
2019-01-21 07:50:00   86622.0  
2019-01-21 07:55:00  236769.0  
2019-01-21 08:00:00  301759.0  
2019-01-21 08:05:00  499644.0  
For episode 0 we get an internal sharpe ratio of nan
For episode 1 we get an internal sharpe ratio of nan
For episode 2 we get an internal sharpe ratio of nan
For episode

For episode 61 we get an internal sharpe ratio of nan
For episode 62 we get an internal sharpe ratio of nan
For episode 63 we get an internal sharpe ratio of nan
For episode 64 we get an internal sharpe ratio of nan
For episode 65 we get an internal sharpe ratio of nan
For episode 66 we get an internal sharpe ratio of nan
For episode 67 we get an internal sharpe ratio of -0.08010497945159843
For episode 68 we get an internal sharpe ratio of nan
For episode 69 we get an internal sharpe ratio of nan
For episode 70 we get an internal sharpe ratio of nan
For episode 71 we get an internal sharpe ratio of nan
For episode 72 we get an internal sharpe ratio of nan
For episode 73 we get an internal sharpe ratio of 0.016007717861910742
For episode 74 we get an internal sharpe ratio of 0.12252282098994613
For episode 75 we get an internal sharpe ratio of -0.05902813266892501
For episode 76 we get an internal sharpe ratio of nan
For episode 77 we get an internal sharpe ratio of 0.05902813361009553

For episode 81 we get an internal sharpe ratio of nan
For episode 82 we get an internal sharpe ratio of nan
For episode 83 we get an internal sharpe ratio of nan
For episode 84 we get an internal sharpe ratio of -0.05607721540920443
For episode 85 we get an internal sharpe ratio of -0.07616100014146915
For episode 86 we get an internal sharpe ratio of nan
For episode 87 we get an internal sharpe ratio of nan
For episode 88 we get an internal sharpe ratio of 0.00029900807095269236
For episode 89 we get an internal sharpe ratio of 0.056077215409204434
For episode 90 we get an internal sharpe ratio of 0.018434882651575946
For episode 91 we get an internal sharpe ratio of nan
For episode 92 we get an internal sharpe ratio of nan
For episode 93 we get an internal sharpe ratio of nan
For episode 94 we get an internal sharpe ratio of 0.025623842532278274
For episode 95 we get an internal sharpe ratio of nan
For episode 96 we get an internal sharpe ratio of nan
For episode 97 we get an interna

For episode 0 we get an internal sharpe ratio of nan
For episode 1 we get an internal sharpe ratio of 0.02696862741277301
For episode 2 we get an internal sharpe ratio of -0.22257091949600996
For episode 3 we get an internal sharpe ratio of -0.22257091949600996
For episode 4 we get an internal sharpe ratio of -0.24597886776276068
For episode 5 we get an internal sharpe ratio of -0.22257091949600996
For episode 6 we get an internal sharpe ratio of -0.22257091949600996
For episode 7 we get an internal sharpe ratio of -0.22257091949600996
For episode 8 we get an internal sharpe ratio of -0.22257091949600996
For episode 9 we get an internal sharpe ratio of -0.22257091949600996
For episode 10 we get an internal sharpe ratio of -0.1105099349084339
For episode 11 we get an internal sharpe ratio of nan
For episode 12 we get an internal sharpe ratio of nan
For episode 13 we get an internal sharpe ratio of nan
For episode 14 we get an internal sharpe ratio of nan
For episode 15 we get an interna

For episode 59 we get an internal sharpe ratio of -0.23259411412044304
For episode 60 we get an internal sharpe ratio of -0.271345618089245
For episode 61 we get an internal sharpe ratio of -0.271345618089245
For episode 62 we get an internal sharpe ratio of -0.271345618089245
For episode 63 we get an internal sharpe ratio of -0.19316198371797982
For episode 64 we get an internal sharpe ratio of -0.271345618089245
For episode 65 we get an internal sharpe ratio of -0.271345618089245
For episode 66 we get an internal sharpe ratio of -0.271345618089245
For episode 67 we get an internal sharpe ratio of 0.0507595736560745
For episode 68 we get an internal sharpe ratio of 0.07711595846205892
For episode 69 we get an internal sharpe ratio of 0.04230911688178731
For episode 70 we get an internal sharpe ratio of 0.010289080770761171
For episode 71 we get an internal sharpe ratio of 0.06687713671554425
0.15664650887484122




 CVCBTC 2019-01-21 00:59:01.170000 2019-01-23 00:48:27.967000
        

For episode 28 we get an internal sharpe ratio of 0.1518943277815028
For episode 29 we get an internal sharpe ratio of 0.1518943277815028
For episode 30 we get an internal sharpe ratio of 0.1596852716513319
For episode 31 we get an internal sharpe ratio of 0.13776913696207216
For episode 32 we get an internal sharpe ratio of 0.1518943277815028
For episode 33 we get an internal sharpe ratio of 0.1518943277815028
For episode 34 we get an internal sharpe ratio of 0.1518943277815028
For episode 35 we get an internal sharpe ratio of 0.1421630082940585
For episode 36 we get an internal sharpe ratio of 0.13460883224002818
For episode 37 we get an internal sharpe ratio of 0.1518943277815028
For episode 38 we get an internal sharpe ratio of 0.1518943277815028
For episode 39 we get an internal sharpe ratio of 0.1518943277815028
For episode 40 we get an internal sharpe ratio of 0.1518943277815028
For episode 41 we get an internal sharpe ratio of 0.1518943277815028
For episode 42 we get an interna

In [35]:
today_port
fname = to_dt(now).strftime("%Y-%m-%d")+"_trades.csv"
today_port.to_csv("/home/me/Documents/BWA/data/trades/{}".format(fname))

,Open,High,Low,Close,Close time,Volume,qsig
Open time,,,,,,,
2019-01-21 07:41:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,53414.0,NaN
2019-01-21 07:42:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,153002.0,NaN
2019-01-21 07:43:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,190503.0,NaN
2019-01-21 07:44:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,280981.0,NaN
2019-01-21 07:45:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,118062.0,NaN
2019-01-21 07:46:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,43519.0,NaN
2019-01-21 07:47:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,31125.0,NaN
2019-01-21 07:48:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,38053.0,NaN
2019-01-21 07:49:00,0.000008,0.000008,0.000008,0.000008,1.548057e+12,17451.0,NaN
